In [174]:
from keras.models import Sequential # type: ignore
from keras.layers import Dense  # type: ignore
from keras.optimizers import Adam  # type: ignore
import tensorflow as tf
from sklearn.model_selection import train_test_split # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy # type: ignore


In [175]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [176]:
train_data.shape

(42000, 785)

In [177]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
X_train , X_cv ,Y_train ,  Y_cv = train_test_split(train_data.drop('label' , axis = 1) , train_data['label'] , test_size = 0.2 , random_state = 42)

In [179]:
X_train.shape


(33600, 784)

In [180]:
X_cv.shape

(8400, 784)

In [181]:
lampda_value = [0.1, 0.5 , 1,5,20,100]
leaning_rate = [0.001, 0.01, 0.1, 0.2 , 0.25]
epoch = [10, 20, 30, 40 , 50]
accurs = []

model = Sequential([
    Dense(2*128, activation='relu', input_shape=(784,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='linear')
])
model.compile(optimizer=Adam(learning_rate=0.001), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history =model.fit(X_train, Y_train, epochs=70)

print(history.history)





c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7841 - loss: 5.4727
Epoch 2/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9245 - loss: 0.3186
Epoch 3/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9425 - loss: 0.2084
Epoch 4/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9519 - loss: 0.1700
Epoch 5/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9576 - loss: 0.1462
Epoch 6/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9607 - loss: 0.1412
Epoch 7/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9690 - loss: 0.1052
Epoch 8/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9727 - loss: 0.0981
Epoch 9/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9738 - loss: 0.0872
Epoch 10/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9795 - loss: 0.0708
Epoch 11/70
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9808 - loss: 0.0725
Epoch 12/70
1050/1050 ━━━━━━━━

In [182]:

result = model.evaluate(X_cv, Y_cv)

y_pred = model.predict(test_data)
sub_data = np.argmax(y_pred, axis=1)
print(sub_data)

submission = pd.DataFrame({'ImageId': range(1, len(sub_data) + 1), 'Label': sub_data})
submission.to_csv('submission.csv', index=False)


263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9762 - loss: 0.3597
875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
[2 0 9 ... 3 9 2]


In [189]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

X_train , X_cv ,Y_train ,  Y_cv = train_test_split(train_data.drop('label' , axis = 1) , train_data['label'] , test_size = 0.2)
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_cv)


params = {
    'max_depth': 3,  
    'eta': 0.001,     
    'objective': 'multi:softmax',  
    'num_class': 10, 
}

num_rounds = 100 
bst = xgb.train(params, dtrain, num_rounds)



In [190]:
y_pred = bst.predict(dtest)


accuracy = accuracy_score(Y_cv, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 75.31%


In [192]:
dtrain = xgb.DMatrix(X_train, label=Y_train)

# Define the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

param_distributions = {
    'n_estimators': [50, 200, 10],
    'max_depth': [3,10, 15],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3]
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=param_distributions,
                                   n_iter=50,  # Number of parameter settings sampled
                                   scoring='accuracy',  # Use accuracy for scoring
                                   cv=3,  # 3-fold cross-validation
                                   verbose=1,
                                   random_state=42)

# Fit RandomizedSearchCV
random_search.fit(X_train, Y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict on the validation set
y_pred = best_model.predict(X_cv)

# Calculate accuracy
accuracy = accuracy_score(Y_cv, y_pred)
print("Best Hyperparameters:", random_search.best_params_)
print("Validation Accuracy:", accuracy)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


c:\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:30:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:32:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:33:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:35:31] WAR